# Compute a Walkable Accessibility Score (WAS) at the block group scale using InfoUSA POI data

In [ ]:
from sklearn.neighbors import BallTree
import numpy as np
import pandas as pd
import geopandas as gpd

## Load POI Data

In [ ]:
#Load 2019 InfoUSA data - other data can be used
# bis = pd.read_csv('2019_Business_Academic_QCQ.txt', sep=",", encoding='latin-1')

bis.head(10)

In [ ]:
#amenities: groceries, restaurants, coffee shops, banks, parks, schools, bookstores, entertainment, and general shopping establishments 
#schools (https://nces.ed.gov/programs/edge/geographic/schoollocations) and parks (centroids - https://www.arcgis.com/home/item.html?id=f092c20803a047cba81fbf1e30eff0b5)
bis['NAICS'] = bis['Primary NAICS Code'].astype(str)
bis['NAICS2'] = bis.NAICS.str[:2]
bis['NAICS4'] = bis.NAICS.str[:4]
bis['NAICS6'] = bis.NAICS.str[:6]
bis.NAICS4.value_counts()

In [ ]:
#specific amenity NAICS codes
ambis = bis.loc[(bis['NAICS2'] == '72') | (bis['NAICS4'] == '4421') | (bis['NAICS4'] == '4431') | (bis['NAICS4'] == '4451') | 
                (bis['NAICS4'] == '4461') | (bis['NAICS4'] == '4481') | (bis['NAICS4'] == '4482') | (bis['NAICS4'] == '4483') |
                (bis['NAICS4'] == '4511') | (bis['NAICS4'] == '4531') | (bis['NAICS4'] == '4532') | (bis['NAICS4'] == '4539') |
                (bis['NAICS4'] == '4453') | (bis['NAICS4'] == '4523') | (bis['NAICS4'] == '5221') | (bis['NAICS6'] == '311811') |
                (bis['NAICS6'] == '451211')]

In [ ]:
ambis = ambis[ambis.Longitude != '-000.000-76']

In [ ]:
#create a geodataframe from coordinates
gbis = gpd.GeoDataFrame(
    ambis, geometry=gpd.points_from_xy(ambis.Longitude, ambis.Latitude), crs='epsg:4326')

In [ ]:
#remove Puerto Rico, Alaska, Hawaii, and US Virgin Islands
gbis = gbis[(gbis['State'] != 'PR') & (gbis['State'] != 'AK') & (gbis['State'] != 'HI') & (gbis['State'] != 'VI')]

In [ ]:
#set CRS
gbis = gbis.to_crs('esri:102003')
gbis.crs

In [ ]:
gbis = gbis[~gbis.is_empty]

In [ ]:
#2011 GreatSchools school data (can use other sources)
sch = gpd.read_file('GreatSchools_2011_us48.shp') 
sch = sch.to_crs('esri:102003')
#2021 ESRI parks data (centroids)
prk = gpd.read_file('Centroids_for_USA_Parks_2021_Buffer2.shp') 
prk = prk.to_crs('esri:102003')

In [ ]:
lst=[gbis,sch,prk]
am=pd.concat(lst, ignore_index=True, axis=0)
am["ID"] = am.index

In [ ]:
am_id = am[['geometry']]
am_id

# Load block groups

In [ ]:
#block group file we're using in this case - one spatial deifnition of demand units for all time periods
s_v = gpd.read_file('BG_2011_2015ADI_us48.shp')
s_v = s_v.set_crs('esri:102003', allow_override=True)
s_v.rename(columns={'GEOID': 'ID'}, inplace=True)
s_v

In [ ]:
#create 5 subsets of continental US BGs
s_v1 = s_v.iloc[0:43167]
s_v1 = s_v1.reset_index(drop=True)
s_v2 = s_v.iloc[43167:86333]
s_v2 = s_v2.reset_index(drop=True)
s_v3 = s_v.iloc[86333:129500]
s_v3 = s_v3.reset_index(drop=True)
s_v4 = s_v.iloc[129500:172666]
s_v4 = s_v4.reset_index(drop=True)
s_v5 = s_v.iloc[172666:215834]
s_v5 = s_v5.reset_index(drop=True)

## Find number of nearest k POI points to each block group

In [ ]:
def get_nearest_neighbors(gdf1, gdf2, k_neighbors=2):
    '''Find k nearest neighbors for all source points from a set of candidate points
    modified from: https://automating-gis-processes.github.io/site/notebooks/L3/nearest-neighbor-faster.html    
    Parameters
    ----------
    gdf1 : geopandas.DataFrame
    Geometries to search from.
    gdf2 : geopandas.DataFrame
    Geoemtries to be searched.
    k_neighbors : int, optional
    Number of nearest neighbors. The default is 2.
    Returns
    -------
    gdf_final : geopandas.DataFrame
    gdf1 with distance, index and all other columns from gdf2.'''

    src_points = [(x,y) for x,y in zip(gdf1.geometry.x , gdf1.geometry.y)]
    candidates =  [(x,y) for x,y in zip(gdf2.geometry.x , gdf2.geometry.y)]

    # Create tree from the candidate points
    tree = BallTree(candidates, leaf_size=15, metric='euclidean')

    # Find closest points and distances
    distances, indices = tree.query(src_points, k=k_neighbors)

    # Transpose to get distances and indices into arrays
    distances = distances.transpose()
    indices = indices.transpose()

    closest_gdfs = []
    for k in np.arange(k_neighbors):
        gdf_new = gdf2.iloc[indices[k]].reset_index()
        gdf_new['distance'] =  distances[k]
        gdf_new = gdf_new.add_suffix(f'_{k+1}')
        closest_gdfs.append(gdf_new)
    
    closest_gdfs.insert(0,gdf1)    
    gdf_final = pd.concat(closest_gdfs,axis=1)

    return gdf_final

In [ ]:
#find closest k amenities for each BG and get also the distance based on Euclidean distance
#whole US subsets
closest_am1 = get_nearest_neighbors(s_v1, am_id, k_neighbors=150)
closest_am2 = get_nearest_neighbors(s_v2, am_id, k_neighbors=150)
closest_am3 = get_nearest_neighbors(s_v3, am_id, k_neighbors=150)
closest_am4 = get_nearest_neighbors(s_v4, am_id, k_neighbors=150)
closest_am5 = get_nearest_neighbors(s_v5, am_id, k_neighbors=150)

In [ ]:
#Wide to long
#Whole US subsets
closest_am1["ID2"] = closest_am1.index
closest_l1 = pd.wide_to_long(closest_am1, ["distance_","index_","geometry_"], i="ID2", j="neighbor")

In [ ]:
closest_am2["ID2"] = closest_am2.index
closest_l2 = pd.wide_to_long(closest_am2, ["distance_","index_","geometry_"], i="ID2", j="neighbor")

In [ ]:
closest_am3["ID2"] = closest_am3.index
closest_l3 = pd.wide_to_long(closest_am3, ["distance_","index_","geometry_"], i="ID2", j="neighbor")

In [ ]:
closest_am4["ID2"] = closest_am4.index
closest_l4 = pd.wide_to_long(closest_am4, ["distance_","index_","geometry_"], i="ID2", j="neighbor")

In [ ]:
closest_am5["ID2"] = closest_am5.index
closest_l5 = pd.wide_to_long(closest_am5, ["distance_","index_","geometry_"], i="ID2", j="neighbor")

In [ ]:
#rename to 'eucidean', 'origin', 'dest'
#whole US subsets
closest_l1['origin'] = closest_l1['ID']
closest_l1['dest'] = closest_l1['index_']
closest_l1['euclidean'] = closest_l1['distance_']
closest_l1= closest_l1.reset_index(level=("neighbor",))
cost1 = closest_l1[['euclidean', 'origin', 'dest','neighbor']]
cost1.sort_values(by=['origin','euclidean'],inplace=True)

In [ ]:
closest_l2['origin'] = closest_l2['ID']
closest_l2['dest'] = closest_l2['index_']
closest_l2['euclidean'] = closest_l2['distance_']
closest_l2= closest_l2.reset_index(level=("neighbor",))
cost2 = closest_l2[['euclidean', 'origin', 'dest','neighbor']]
cost2.sort_values(by=['origin','euclidean'],inplace=True)

In [ ]:
closest_l3['origin'] = closest_l3['ID']
closest_l3['dest'] = closest_l3['index_']
closest_l3['euclidean'] = closest_l3['distance_']
closest_l3= closest_l3.reset_index(level=("neighbor",))
cost3 = closest_l3[['euclidean', 'origin', 'dest','neighbor']]
cost3.sort_values(by=['origin','euclidean'],inplace=True)

In [ ]:
closest_l4['origin'] = closest_l4['ID']
closest_l4['dest'] = closest_l4['index_']
closest_l4['euclidean'] = closest_l4['distance_']
closest_l4= closest_l4.reset_index(level=("neighbor",))
cost4 = closest_l4[['euclidean', 'origin', 'dest','neighbor']]
cost4.sort_values(by=['origin','euclidean'],inplace=True)

In [ ]:
closest_l5['origin'] = closest_l5['ID']
closest_l5['dest'] = closest_l5['index_']
closest_l5['euclidean'] = closest_l5['distance_']
closest_l5= closest_l5.reset_index(level=("neighbor",))
cost5 = closest_l5[['euclidean', 'origin', 'dest','neighbor']]
cost5.sort_values(by=['origin','euclidean'],inplace=True)

## Calculate accessibility measure

In [ ]:
# https://journals-sagepub-com.may.idm.oclc.org/doi/10.1177/0265813516641685
#convert distance into time (rate of 5kph)
cost1['time'] = (cost1.euclidean*3600)/5000
cost2['time'] = (cost2.euclidean*3600)/5000
cost3['time'] = (cost3.euclidean*3600)/5000
cost4['time'] = (cost4.euclidean*3600)/5000
cost5['time'] = (cost5.euclidean*3600)/5000

# choose 'upper' parameter (for testing)
# upper = 800
# upper = 1600
# upper = 2400

# choose decay rate
# decay = .005
# decay = .008
# decay = .01

In [ ]:
cost1['LogitT_5'] = 1-(1/(np.e**((upper/180)-decay*cost1.time)+1))
cost2['LogitT_5'] = 1-(1/(np.e**((upper/180)-decay*cost2.time)+1))
cost3['LogitT_5'] = 1-(1/(np.e**((upper/180)-decay*cost3.time)+1))
cost4['LogitT_5'] = 1-(1/(np.e**((upper/180)-decay*cost4.time)+1))
cost5['LogitT_5'] = 1-(1/(np.e**((upper/180)-decay*cost5.time)+1))

In [ ]:
# plt.hist(cost.LogitT_5, bins=50)
# plt.hist(cost1.LogitT_5, bins=50)

In [ ]:
#sum weighted distances by tract (origin) ID
cost_sum1 = cost1.groupby("origin").sum()
cost_sum1['ID'] = cost_sum1.index
cost_sum2 = cost2.groupby("origin").sum()
cost_sum2['ID'] = cost_sum2.index
cost_sum3 = cost3.groupby("origin").sum()
cost_sum3['ID'] = cost_sum3.index
cost_sum4 = cost4.groupby("origin").sum()
cost_sum4['ID'] = cost_sum4.index
cost_sum5 = cost5.groupby("origin").sum()
cost_sum5['ID'] = cost_sum5.index

In [ ]:
cost_merge1 = s_v1.merge(cost_sum1, how='inner', on='ID')
cost_merge2 = s_v2.merge(cost_sum2, how='inner', on='ID')
cost_merge3 = s_v3.merge(cost_sum3, how='inner', on='ID')
cost_merge4 = s_v4.merge(cost_sum4, how='inner', on='ID')
cost_merge5 = s_v5.merge(cost_sum5, how='inner', on='ID')

In [ ]:
#export for given year
# cost_merge1.to_file('us_walkability_access_score_2019_1.shp')
# cost_merge2.to_file('us_walkability_access_score_2019_2.shp')
# cost_merge3.to_file('us_walkability_access_score_2019_3.shp')
# cost_merge4.to_file('us_walkability_access_score_2019_4.shp')
# cost_merge5.to_file('us_walkability_access_score_2019_5.shp')